# Le Debruiteur
* Jonas Freiburghaus
* Romain Capocasale
* He-Arc, INF3dlm-a
* Image Processing course
* 2019-2020

# Import

In [ ]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

from debruiteur.generator.datagenerator import DataGenerator
from debruiteur.models.autoencoders import build_conv_autoencoder, build_dense_autoencoder
from debruiteur.models.gan import GAN
from debruiteur.noise.noise import *
from debruiteur.plots.plots import plot_model_loss
from debruiteur.preprocessing.preprocessor import *
from debruiteur.utils.utils import save_model, split_train_val_df

import os

# Configurations

In [2]:
noise_class_list = [
    GaussianNoise(mean=0, std=10),
    PoissonNoise(),
    UniformNoise(amplitude=100),
    SaltPepperNoise(freq=0.3),
    SquareMaskNoise(mask_shape=(10, 10), freq=0.1),
    SpeckleNoise(),
    AveragingBlurNoise(),
    GaussianBlurNoise(),
    MedianBlurNoise()
]

# Create Dataframe

In [6]:
working_dir = os.path.abspath(os.getcwd())

df_original = make_original_dataframe(os.path.join(working_dir, "images"))
df_resized = make_resized_dataframe(df_original, img_shape=(64, 64, 1), resized_path=os.path.join(working_dir, "resized_images"))
df_noised = make_noised_dataframe(df_resized, noise_class_list, os.path.join(working_dir, "noised_images"))
df_train, df_val = split_train_val_df(df_noised)

# Models training

## Data generator

In [ ]:
train_gen = DataGenerator(df_train)
val_gen = DataGenerator(df_val)

## Convolutional autoencoder

In [ ]:
ae_model = build_conv_autoencoder()
ae_model.summary()

In [ ]:
history = ae_model.fit_generator(train_gen, validation_data=val_gen, epochs=10)
plot_model_loss(history)

In [ ]:
save_model(ae_model, os.path.join(working_dir, "saved_models"), "conv_autoencoder.h5")

del ae_model
del train_gen
del val_gen

## Dense autoencoder

In [ ]:
train_gen = DataGenerator(df_train, img_shape=(10000, ))
val_gen = DataGenerator(df_val, img_shape=(10000, ))

In [ ]:
ae_model = build_dense_autoencoder(loss="mse")
ae_model.summary()

In [ ]:
history = ae_model.fit_generator(train_gen, validation_data=val_gen, epochs=10)
plot_model_loss(history)

In [ ]:
save_model(ae_model, os.path.join(working_dir, "saved_models"), "dense_autoencoder.h5")

del ae_model
del train_gen
del val_gen

## GAN autoencoder

In [7]:
train_gen = DataGenerator(df_train)
val_gen = DataGenerator(df_val)

In [8]:
gan = GAN()

ValueError: Operands could not be broadcast together with shapes (100, 100, 16) (100, 100, 32)

In [ ]:
gan.generator.summary()

In [ ]:
gan.discriminator.summary()

In [ ]:
train_hist, val_hist = gan.train(train_gen, val_gen)